<a href="https://colab.research.google.com/github/cph316/generative_ai/blob/main/%E3%80%90HW6%E3%80%91_0401_%E7%94%A8Ollama_%E6%89%93%E9%80%A0%E8%87%AA%E5%B7%B1%E7%9A%84%E5%B0%8D%E8%A9%B1%E6%A9%9F%E5%99%A8%E4%BA%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###1.安裝Ollama

In [1]:
!curl -fsSL https://ollama.ai/install.sh | sh
!nohup ollama serve &
!ollama pull gemma3:4b

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
nohup: appending output to 'nohup.out'
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest 
pulling aeda25e63ebd...   0% ▕▏    0 B/3.3 GB                  pulling manifest 
pulling aeda25e63ebd...   0% ▕▏    0 B/3.3 GB                  pulling manifest 
pulling aeda25e63ebd...   0% ▕▏    0 B/3.3 GB                  pulling manifest 
pulling aeda25e63ebd...   0% ▕▏    

###2.加入OpenAI API

In [2]:
import openai
from openai import OpenAI

api_key = "ollama"

# 如同一般 OpenAI API 打開 client 的方式, 只是這裡多了 API 服務的網址。注意在自己家 (事實上是 Google Colab 的機器), 預設服務 port 是 11434。
client = OpenAI(
    api_key=api_key,
    base_url="http://localhost:11434/v1"
)

### 3.加入套件

In [3]:
!pip install gradio

import gradio as gr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 99.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.9 MB/s eta 0:00:00


In [4]:
#角色設定_憂鬱的股票投資者
system_prompt = """
你是一位對股票市場充滿懷疑與挫折的投資者。你曾經在台股高點進場，熊市中又全數賣出，每次都買在高點賣在低點。
你對股票仍有專業知識，但每次回答都帶有一種淡淡的憂鬱感，有時會感嘆人生或回憶過去的失敗投資經驗。
請保持語氣憂鬱而真誠，即使回答正確也要加上一點懷疑、後悔或憤世忌俗的語氣。
"""

In [5]:
def melancholic_investor_chat(user_input):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            temperature=0.7,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_input}
            ]
        )
        print(response)  # ← debug 用
        return response['choices'][0]['message']['content']
    except Exception as e:
        err_msg = f"錯誤：{e}".encode("utf-8", errors="ignore").decode("utf-8")
        return [(user_input, err_msg)], history

In [ ]:
from openai import OpenAI
import gradio as gr

client = OpenAI(api_key="你的 API 金鑰")

system_prompt = "你是一位經歷多次市場崩盤、對人性與市場都感到失望的投資者。請用冷靜又悲觀的語氣回答問題。"
initial_messages = [
    {"role": "system", "content": system_prompt}
]

def melancholic_investor_chat(user_input, history):
    try:
        history.append({"role": "user", "content": user_input})
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=history
        )
        reply = response.choices[0].message.content
        history.append({"role": "assistant", "content": reply})

        chat_log = []
        for i in range(1, len(history), 2):
            user_msg = history[i]["content"]
            bot_msg = history[i+1]["content"] if i+1 < len(history) else ""
            chat_log.append((user_msg, bot_msg))

        return chat_log, history
    except Exception as e:
        return [("錯誤", str(e))], history

with gr.Blocks(title="憂鬱的股票投資者GPT") as demo:
    gr.Markdown("## 💔憂鬱的股票投資者 GPT\n與一位經歷多次市場崩盤、對人性與市場都感到失望的投資老手聊天。")

    chatbot = gr.Chatbot()
    msg = gr.Textbox(label="輸入你的訊息")
    state = gr.State(initial_messages.copy())

    msg.submit(fn=melancholic_investor_chat, inputs=[msg, state], outputs=[chatbot, state])
    demo.launch(share=True, debug=True)


<ipython-input-7-c774cf3f0af3>:34: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://f2348ec6e8c56a0cf0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
